In [29]:
# ---------------------------------------------------------
# Initialize Spark Session
# ---------------------------------------------------------
# Purpose:
# - Create a Spark entry point for DataFrame operations
# - Optimize for local Docker execution by reducing shuffle partitions
# ---------------------------------------------------------

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .appName("NYC_Jobs_Data_Exploration") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

spark

In [30]:
# ---------------------------------------------------------
# Load NYC Jobs CSV Dataset
# ---------------------------------------------------------
# Assumptions:
# - CSV file is placed under /workspace/data/
# - Header is present
# - Schema inference is enabled for initial exploration
# ---------------------------------------------------------

df_raw = spark.read \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv("../data/nyc-jobs.csv")

# Preview sample records
# df_raw.show(5, truncate=False)

In [31]:
df_raw.printSchema()

root
 |-- Job ID: integer (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Posting Type: string (nullable = true)
 |-- # Of Positions: integer (nullable = true)
 |-- Business Title: string (nullable = true)
 |-- Civil Service Title: string (nullable = true)
 |-- Title Code No: string (nullable = true)
 |-- Level: string (nullable = true)
 |-- Job Category: string (nullable = true)
 |-- Full-Time/Part-Time indicator: string (nullable = true)
 |-- Salary Range From: double (nullable = true)
 |-- Salary Range To: double (nullable = true)
 |-- Salary Frequency: string (nullable = true)
 |-- Work Location: string (nullable = true)
 |-- Division/Work Unit: string (nullable = true)
 |-- Job Description: string (nullable = true)
 |-- Minimum Qual Requirements: string (nullable = true)
 |-- Preferred Skills: string (nullable = true)
 |-- Additional Information: string (nullable = true)
 |-- To Apply: string (nullable = true)
 |-- Hours/Shift: string (nullable = true)
 |-- Work Locat

In [33]:
# ---------------------------------------------------------
# Standardize Column Names
# ---------------------------------------------------------
# Purpose:
# - Convert space-separated column names to snake_case
# - Improve readability and prevent quoting/backticks
# - Prepare dataset for clean KPI logic
# ---------------------------------------------------------

def standardize_column_names(df):
    """
    Converts column names to snake_case by:
    - Trimming leading/trailing spaces
    - Converting to lowercase
    - Replacing spaces with underscores
    """
    for c in df.columns:
        standardized_name = (
            c.strip()
             .lower()
             .replace(" ", "_")
        )
        df = df.withColumnRenamed(c, standardized_name)
    return df
df = standardize_column_names(df_raw)
df.printSchema()

root
 |-- job_id: integer (nullable = true)
 |-- agency: string (nullable = true)
 |-- posting_type: string (nullable = true)
 |-- #_of_positions: integer (nullable = true)
 |-- business_title: string (nullable = true)
 |-- civil_service_title: string (nullable = true)
 |-- title_code_no: string (nullable = true)
 |-- level: string (nullable = true)
 |-- job_category: string (nullable = true)
 |-- full-time/part-time_indicator: string (nullable = true)
 |-- salary_range_from: double (nullable = true)
 |-- salary_range_to: double (nullable = true)
 |-- salary_frequency: string (nullable = true)
 |-- work_location: string (nullable = true)
 |-- division/work_unit: string (nullable = true)
 |-- job_description: string (nullable = true)
 |-- minimum_qual_requirements: string (nullable = true)
 |-- preferred_skills: string (nullable = true)
 |-- additional_information: string (nullable = true)
 |-- to_apply: string (nullable = true)
 |-- hours/shift: string (nullable = true)
 |-- work_locat

In [34]:
# ---------------------------------------------------------
# KPI 1: Top 10 Job Categories by Number of Job Postings
# ---------------------------------------------------------
# Logic:
# - Group data by job_category
# - Count number of job postings
# - Sort by count in descending order
# - Select top 10 categories
# ---------------------------------------------------------

from pyspark.sql.functions import col

kpi_1_jobs_per_category = (
        df
        .groupBy("job_category")
        .count()
        .orderBy(col("count").desc())
        .limit(10)
)

kpi_1_jobs_per_category.show(truncate=False)


+-----------------------------------------+-----+
|job_category                             |count|
+-----------------------------------------+-----+
|Engineering, Architecture, & Planning    |504  |
|Technology, Data & Innovation            |313  |
|Legal Affairs                            |226  |
|Public Safety, Inspections, & Enforcement|182  |
|Building Operations & Maintenance        |181  |
|Finance, Accounting, & Procurement       |169  |
|Administration & Human Resources         |134  |
|Constituent Services & Community Programs|129  |
|Health                                   |125  |
|Policy, Research & Analysis              |124  |
+-----------------------------------------+-----+



In [35]:
# ---------------------------------------------------------
# Create Average Salary Feature
# ---------------------------------------------------------
# Purpose:
# - Combine salary_range_from and salary_range_to
# - Create a single salary metric for all KPI calculations
# - Avoid repeating salary math in every KPI
# ---------------------------------------------------------

from pyspark.sql.functions import col, round

df = df.withColumn(
    "avg_salary",
    round(
        (col("salary_range_from") + col("salary_range_to")) / 2,
        2
    )
)


In [36]:
# ---------------------------------------------------------
# KPI 2: Salary Distribution per Job Category
# ---------------------------------------------------------

kpi_2_salary_distribution = (
    df
        .groupBy("job_category")
        .agg(
            round(min("avg_salary"), 2).alias("min_salary"),
            round(avg("avg_salary"), 2).alias("avg_salary"),
            round(max("avg_salary"), 2).alias("max_salary")
        )
        .orderBy(col("avg_salary").desc())
)

kpi_2_salary_distribution.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+----------+----------+
|job_category                                                                                                                                                                                             |min_salary|avg_salary|max_salary|
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+----------+----------+
|Administration & Human Resources Finance, Accounting, & Procurement Building Operations & Maintenance                                                                                                    |218587.0  |218587.0  |218587.0  |
|Engineering, Architecture, & Planning Maintenance &

In [38]:
# ---------------------------------------------------------
# Create requires_degree Feature
# ---------------------------------------------------------
# Purpose:
# - Identify whether a job posting requires a higher degree
# - Enable correlation analysis between education and salary
# - Derived from minimum qualification text
# ---------------------------------------------------------

from pyspark.sql.functions import col

df = df.withColumn(
    "requires_degree",
    col("minimum_qual_requirements").rlike(
        "Bachelor|Master|PhD|Degree"
    )
)

In [39]:
# ---------------------------------------------------------
# KPI 3: Degree Requirement vs Salary
# ---------------------------------------------------------

kpi_3_degree_vs_salary = (
    df
        .groupBy("requires_degree")
        .agg(
            round(avg("avg_salary"), 2).alias("average_salary"),
            count("*").alias("job_count")
        )
)

kpi_3_degree_vs_salary.show(truncate=False)

+---------------+--------------+---------+
|requires_degree|average_salary|job_count|
+---------------+--------------+---------+
|null           |82481.61      |18       |
|false          |71803.21      |2728     |
|true           |76980.44      |200      |
+---------------+--------------+---------+



In [40]:
# ---------------------------------------------------------
# KPI 4: Highest Paying Job per Agency
# ---------------------------------------------------------

from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

agency_window = Window.partitionBy("agency").orderBy(col("avg_salary").desc())

kpi_4_highest_salary_per_agency = (
    df
        .withColumn("rank", row_number().over(agency_window))
        .filter(col("rank") == 1)
        .select("agency", "job_category", "avg_salary")
)

kpi_4_highest_salary_per_agency.show(truncate=False)


+-----------------------------+------------------------------------------------------------------------------------------------------------------------------------------+----------+
|agency                       |job_category                                                                                                                              |avg_salary|
+-----------------------------+------------------------------------------------------------------------------------------------------------------------------------------+----------+
|ADMIN FOR CHILDREN'S SVCS    |Finance, Accounting, & Procurement                                                                                                        |117474.5  |
|ADMIN TRIALS AND HEARINGS    |Administration & Human Resources Constituent Services & Community Programs Legal Affairs                                                  |90000.0   |
|BOARD OF CORRECTION          |Technology, Data & Innovation Policy, Research & Analysis  

In [42]:
# ---------------------------------------------------------
# Parse Posting Date
# ---------------------------------------------------------
# Purpose:
# - Convert posting_date from string to DateType
# - Enable time-based filtering and aggregations
# ---------------------------------------------------------

from pyspark.sql.functions import to_date

df = df.withColumn(
    "posting_date",
    to_date(col("posting_date"), "MM/dd/yyyy")
)

In [43]:
# ---------------------------------------------------------
# Create posting_year Feature
# ---------------------------------------------------------
# Purpose:
# - Extract year from posting_date
# - Support time-based KPIs (e.g., last 2 years analysis)
# ---------------------------------------------------------

from pyspark.sql.functions import year

df = df.withColumn(
    "posting_year",
    year(col("posting_date"))
)


In [45]:
# ---------------------------------------------------------
# KPI 5: Average Salary per Agency for the Last 2 Years
# ---------------------------------------------------------

from pyspark.sql.functions import avg, round, count

current_year = spark.sql(
    "SELECT year(current_date())"
).collect()[0][0]

kpi_5_avg_salary_last_2_years = (
    df
        .filter(col("posting_year") >= current_year - 2)
        .groupBy("agency")
        .agg(
            round(avg(col("avg_salary")), 2).alias("avg_salary_last_2_years"),
            count("*").alias("job_count")
        )
        .orderBy(col("avg_salary_last_2_years").desc())
)

kpi_5_avg_salary_last_2_years.show(truncate=False)


+------+-----------------------+---------+
|agency|avg_salary_last_2_years|job_count|
+------+-----------------------+---------+
+------+-----------------------+---------+



In [46]:
# ---------------------------------------------------------
# KPI 6: Highest Paid Skills
# ---------------------------------------------------------

from pyspark.sql.functions import explode, split, trim

kpi_6_highest_paid_skills = (
    df
        .filter(col("minimum_qual_requirements").isNotNull())
        .withColumn(
            "skill",
            explode(split(col("minimum_qual_requirements"), ","))
        )
        .withColumn("skill", trim(col("skill")))
        .groupBy("skill")
        .agg(
            round(avg("avg_salary"), 2).alias("avg_salary"),
            count("*").alias("job_count")
        )
        .filter(col("job_count") >= 10)
        .orderBy(col("avg_salary").desc())
        .limit(20)
)

kpi_6_highest_paid_skills.show(truncate=False)


+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+---------+
|skill                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                